# Solar Power

In [What Are Orbital Rings?](/what-are-orbital-rings.html) we briefly talked about the possibility for orbital rings to provide extremely cost-effective solar power, but not how or why. Let's dive into that.

## Terrestrial vs Space-Based Solar

Terrestrial solar has the advantage of being, well, on the ground; we don't need to send the panels into space, which is extremely expensive right now. However, it has a few serious disadvantages:

- About 30% of solar radiation reflects off the atmosphere, and another 23% is absorbed before it hits the ground. That means only about 47% of the sun's light actually reaches the surface. (XXX Cite: https://www.ucdavis.edu/climate/definitions/how-is-solar-power-generated#:~:text=About%2030%20percent%20of%20incoming,atmospheric%20particles%2C%20dust%20and%20ozone.)
- Weather causes a drastic impact on its efficiency, causing 30-50% reductions due to cloud coverage and 10-20% in heavy rain. (XXX: Add citation)
- Weather also necessitates more durable panels, which increases their weight and cost (due to additional thickness of the glass layer) (XXX: Add citation)
- Solar irradiance is highly dependent on latitude, with the highest intensity at the equator and falling off as you get closer to the poles (XXX: Add citation)

Space-based solar has none of these problems, though it does carry a few notable ones of its own:

- It's currently very expensive to send mass quantities of solar panels into space, and getting the energy back down to Earth is both difficult and inefficient (XXX: Add citation)
- Panels in normal Earth orbits experience drastic temperature swings many times per day, which impact both the durability and effiency (XXX: Add citation)

However, neither of these problems are inherent to space-based solar, just the implementations we have at hand right now. Let's talk about how orbital rings can solve both of these.

## Solar On Orbital Rings

Orbital rings provide a unique set of solutions to this problem. Getting the solar panels up there is as easy as taking an electric lift from the ground up to space, and they only experience the temperature swing once a day. Since solar panels are more efficient when they're colder (XXX: Add citation), we *could* build a system to distribute heat from the hot side to the cold side, evening it out over the course of a day; however, we'll keep that out of scope for now.

## Efficiency

Outside of (the bulk of) Earth's atmosphere, the solar irradiance is about $1,360 W/m^2$. That means every square meter of Earth's atmosphere receives 1360 watts of energy from the sun. We can't capture all of that, but with high-efficiency panels, we can capture about 30%. That means a 1 square meter solar panel on an orbital ring, pointed straight at the sun, would produce 408 watts.

Now, imagine we have solar panels all the way around an orbital ring. How many of those are pointed straight at the sun, receiving the full irradiance? Well, we know that half of them are going to be on the other side of the Earth, so we immediately cut our power per square meter in half. But we also need to account for the ones that are on the sun-facing side, but angled away.

In this interactive widget, you can play around with a simple model and see what percentage of light is being converted on average by each panel.

In [1]:
from IPython.display import display, HTML
display(HTML('''
<canvas id="solarcanvas" width=500 height=500></canvas>
<div>Number of panels <input type="range" min="0" max="1" value="0.5" step="0.01" id="num-panels"> <span id="num-panels-text"></span></div>
<div>Average illumination: <span id="avg-illumination"></span>%</div>
<script>
(() => {
    class Vector2 {
      constructor(x, y) {
        this.x = x
        this.y = y
      }
    
      add(vector) {
        return new Vector2(this.x + vector.x, this.y + vector.y)
      }
    
      subtract(vector) {
        return new Vector2(this.x + vector.x, this.y + vector.y)
      }
    
      multiply(scalar) {
        return new Vector2(this.x * scalar, this.y * scalar)
      }
    
      dot(vector) {
        return this.x * vector.x + this.y * vector.y
      }

      length() {
          return Math.sqrt(this.x * this.x + this.y * this.y)
      }

      normalize() {
          const len = this.length()
          return new Vector2(this.x / len, this.y / len)
      }
    
      rotate(angle) {
        const cos = Math.cos(angle)
        const sin = Math.sin(angle)
        return new Vector2(
          this.x * cos - this.y * sin,
          this.x * sin + this.y * cos
        )
      }
    }
    
    const cvs = document.getElementById('solarcanvas')
    const ctx = cvs.getContext('2d')
    const range = document.getElementById('num-panels')
    const numPanelsElement = document.getElementById('num-panels-text')
    const avgIllum = document.getElementById('avg-illumination')
    const render = () => {
        const numPanels = Math.round(3 * Math.pow(1200, parseFloat(range.value)))
        numPanelsElement.innerText = numPanels
        const radius = 100
        const distFromCircle = 10
        const thickness = 10
        const halfSideLength = Math.tan(Math.PI / numPanels) * (radius + distFromCircle)
        ctx.fillStyle = '#ddf'
        ctx.fillRect(0, 0, 500, 500)
        ctx.beginPath()
        ctx.arc(250, 250, radius, 0, 2 * Math.PI)
        ctx.strokeStyle = 'black'
        ctx.stroke()
        const sunPos = new Vector2(1000000, 1000000)
        const rot = 2 * Math.PI / numPanels
        const start = new Vector2(0, 1)
        const topRight = new Vector2(halfSideLength, thickness / 2)
        const bottomLeft = topRight.multiply(-1)
        const topLeft = new Vector2(bottomLeft.x, topRight.y)
        const bottomRight = new Vector2(topRight.x, bottomLeft.y)

        const drawRect = (a, b, c, d) => {
            ctx.beginPath()
            ctx.moveTo(a.x, a.y)
            ctx.lineTo(b.x, b.y)
            ctx.lineTo(c.x, c.y)
            ctx.lineTo(d.x, d.y)
            ctx.closePath()
            ctx.fill()
        }
        let totalIllum = 0
        for(let i = 0; i < numPanels; ++i) {
            const theta = i * rot
            const pos = start.rotate(theta)
            const normal = pos.normalize()
            const toLight = sunPos.subtract(pos).normalize()
            const angle = -normal.dot(toLight)
            if(angle <= 0)
                ctx.fillStyle = 'black'
            else {
                const color = Math.round(angle * 255)
                ctx.fillStyle = `rgb(${color} ${color} ${color})`
                totalIllum += angle
            }
            const center = pos.multiply(radius + distFromCircle + thickness / 2).add(new Vector2(250, 250))
            drawRect(
                bottomLeft.rotate(theta).add(center),
                topLeft.rotate(theta).add(center),
                topRight.rotate(theta).add(center),
                bottomRight.rotate(theta).add(center)
            )
        }
        avgIllum.innerText = totalIllum / numPanels * 100
    }
    render()
    range.addEventListener('input', render)
})()
</script>'''))

As we can see, with more and more panels, this converges to about 31.83%. For the mathematically inclined, there's a simple integral to find this:

$$facingTheSun = \int_{0}^{1} \frac{sin(x\pi)}{2} \,dx = \frac{1}{\pi} \approx 0.31831$$

By multiplying this by our original 408 watts per square meter, we find an average production of 130 watts per square meter.

## Bringing It All Together

Now, how much power do we actually need to generate? Well, the world energy consumption was 24,398 TWh (tera-watt hours, or trillions of watt-hours, a measure of energy usage), which -- when we divide that by a year -- means that at any given time, we averaged a consumption of 2.785 TW of power.

However, power consumption can vary drastically; while we're looking at the whole world, our populations and energy consumption aren't evenly distributed. It's hard to get a solid figure for what our peak consumption would look like, and there will certainly be losses and inefficiencies in our system, so we'll need to fudge the numbers a bit here. Let's assume we need to generate an additional 30%. That means we need enough panels to produce 3.6205 TW.

How much area do we need to cover in solar panels, given that?

$$Area = \frac{3.6205 * 10^{12} W}{130 W/m^2} = 2.785 * 10^{10} m^2 = 27850 km^2$$

This is not a small area, by any stretch of the imagination. In fact, it's about the size of Massachusetts! However, with the set of 15 orbital rings we'll be discussing today, this would only require that the rings be 45 meters wide, to blanket the entire world in cheap power. But speaking of price...

### Cost

This is where things get just a little tricky. Figuring out the cost for solar panels is straightforward on the scale of a house, a warehouse, or even a local solar power plant; figuring out the cost for a quantity of solar panels that outstrips the total worldwide market by leaps and bounds is -- to put it mildly -- more complicated.

In all likelihood, someone wanting to build this system of orbital rings would, themselves, become the biggest producer of solar panels in the world. This means that the cost will be a function of the costs of: the materials, production facilities, labor, and shipping. While we could work this out by figuring out the profit margins of existing producers, we also need to consider the economies of scale, and that our panels would be different (thinner top glass, for instance) from those used terrestrially. Any number I could throw out here would be an educated guess at the absolute best, and a complete fabrication at worst.

So instead, let's use the cost of typical solar panels we use here on the ground. These can vary drastically from about $50-200 per square meter, so we'll stick to the low end of the range. On the ground, that would typically get us a low-efficiency panel (and thus require more space), but let's hope that our economies of scale get us those nice high-efficiency panels for these prices. Worst case, we can always make our rings wider.

Regardless, working this one out is straightforward: we multiply the area by the price per area.

$$ 27850 km^2 * $50/m^2 = $1.3925\ trillion $$

Okay, so clearly this isn't going to be a cheap endeavor. But I promised low costs at the beginning.

## Putting It In Perspective

The worldwide average cost for electricity in 2020 was \$0.14 per kilowatt-hour for households and \$0.12 for businesses; let's take the lower cost to estimate the total prices paid by consumers. Taking \$0.12/kWh and multiplying it by the 24,398 TWh worldwide energy consumption gives us a total of \$2.93 trillion dollars.

Our solar panels might cost us \$1.3925 trillion, but that's a one-time cost. If we hooked everyone on the ground up to our power and charged them \$0.06/kWh -- half the price businesses are paying now -- our solar panels would pay for themselves in under a year.

## Cost To Consumers

While it sounds nice to repay our investment in the solar panels in a year, let's make a more reasonable estimate for what we should actually charge customers. First, some assumptions we'll make:

- The solar panels last, on average, 10 years each. This is almost certainly very low, with realistic estimates coming in between 25 and 50 years, but 10 years makes the math simpler and doesn't affect things too much in the end.
- Power consumption will remain steady year-over-year. We know this is wrong, as it is already going up every year, and cheaper power will almost certainly mean people will use more, but the costs here scale linearly, so it doesn't really matter for our calculations, except when it comes to determining profits.
- Upkeep (aside from the cost of new/replacement solar panels) and other infrastructural costs are $100 billion per year. This number is a complete fabrication, but provides a starting point.
- We'll aim to repay our solar panels over the course of 5 years. This is still on the short side of repayment for major investments in infrastructure, but strikes a good balance for consumer cost versus our investment.
- We'll have a 10% profit margin to reinvest.

When we run the math for this, we find that the cost to consumers would be **\$0.0233/kWh** -- an **80% reduction** in worldwide average energy costs! We'd profit nearly \$52 billion a year, on top of that. However, if you find some of these assumptions out of line, here's an interactive widget to try it out for yourself and see how changing them affects the costs:

In [13]:
from IPython.display import display, HTML
display(HTML('''
<div>Average lifespan of solar panels: <input type="range" min="1" max="50" value="10" step="0.1" id="lifespan"> <span id="lifespan-label"></span> years</div>
<div>Upkeep per year: <input type="range" min="0" max="250" value="100" step="0.1" id="upkeep"> $<span id="upkeep-label"></span> billion per year</div>
<div>Time to repay our investment: <input type="range" min="1" max="25" value="5" step="0.1" id="repay"> <span id="repay-label"></span> years</div>
<div>Profit margin: <input type="range" min="0" max="200" value="10" step="0.1" id="margin"> <span id="margin-label"></span>%</div>
<hr>
<div>Cost to consumers: <span style="font-weight: bold">$<span id="cost-label"></span>/kWh</span></div>
<div>Profit: $<span id="profit-label"></span> billion per year</span></div>
<script>
(() => {
    const lifespanElem = document.getElementById('lifespan')
    const lifespanLabel = document.getElementById('lifespan-label')
    const upkeepElem = document.getElementById('upkeep')
    const upkeepLabel = document.getElementById('upkeep-label')
    const repayElem = document.getElementById('repay')
    const repayLabel = document.getElementById('repay-label')
    const marginElem = document.getElementById('margin')
    const marginLabel = document.getElementById('margin-label')
    const costLabel = document.getElementById('cost-label')
    const profitLabel = document.getElementById('profit-label')
    const update = () => {
        lifespanLabel.innerText = lifespanElem.value
        upkeepLabel.innerText = upkeepElem.value
        repayLabel.innerText = repayElem.value
        marginLabel.innerText = marginElem.value
        
        const lifespan = parseFloat(lifespanElem.value) // in years
        const upkeep = parseFloat(upkeepElem.value) * 1e9 // in USD/year
        const repay = parseFloat(repayElem.value) // in years
        const margin = parseFloat(marginElem.value) / 100

        const production = 24398e9; // in kWh -- this is technically low because we added a fudge factor for spikes
        const initialCost = 1.3925e12; // in USD
        const yearlyCost = initialCost / repay + initialCost / lifespan + upkeep // in USD
        const profit = yearlyCost * margin // in USD
        const withProfit = yearlyCost + profit // in USD
        const cost = withProfit / production // in $/kWh
        costLabel.innerText = Math.round(cost * 10000) / 10000
        const profitBillions = profit / 1e9
        profitLabel.innerText = Math.round(profitBillions * 100) / 100
    }
    update()
    lifespanElem.addEventListener('input', update)
    upkeepElem.addEventListener('input', update)
    repayElem.addEventListener('input', update)
    marginElem.addEventListener('input', update)
})()
</script>'''))